## 最終課題

In [7]:
#webスクレイピングに最低限必要なライブラリをインポート
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# --- 設定 ---
#最初に参照するURLを設定する
START_URL = "https://www.musashino-u.ac.jp/"
#対象とするドメイン名を取得する
BASE_NETLOC = urlparse(START_URL).netloc

# --- データ構造 ---
# 取得した結果を格納する辞書 {URL: <title>}として表示する
scraped_data = {}

# 巡回キュー (これから確認するURLの集合)
to_visit = {START_URL}

# 既に確認済みのURLの集合
visited = set()

# --- 関数: ページの情報を取得し、次のリンクを見つける ---
def crawl_page(url, scraped_data, to_visit, visited):
    """単一のURLを処理し、タイトルを取得し、新しいリンクをto_visitに追加する"""
    
    # 既に処理済みのURLはスキップできるようにする
    if url in visited:
        return

    print(f"{url}")
    visited.add(url)
    time.sleep(0.3) # サーバーへの負荷軽減をするために1秒待機させる

    #tryを使用することで、エラーが発生した場合にプログラムがクラッシュしないようにする
    #→exceptブロックでエラーメッセージを表示し、関数を終了する
    try:
        # ページのHTMLを取得 
        #サーバーから応答が返ってくるまでプログラムが無限に待機しないように、タイムアウトを10秒に設定する
        response = requests.get(url, timeout=10)
        # 成功ステータスのみ処理
        response.raise_for_status() 
        #ページのエンコーディングを自動で判別して設定できるようにする
        response.encoding = response.apparent_encoding 
    except requests.RequestException as e:
        print(f"  [エラー]: {e}")
        return

    # 取得したHTMLをBeautifulSoupを使い解析できる形に変換して解析
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # --- <title>の抽出と辞書への格納 ---
    title_tag = soup.find('title')
    title = title_tag.text.strip() if title_tag else "タイトルなし"
    
    # 辞書に {URL: <title>} の形式で格納
    scraped_data[url] = title
    print(f" タイトル: {title}")

    # --- リンクの発見 ---
    for link in soup.find_all('a', href=True):
        href = link['href']
        
        # 相対パスをURLに変換し、フラグメント（#）を除去
        absolute_url = urljoin(url, href).split('#')[0]
        
        # URLを解析してドメイン情報をチェック
        parsed_url = urlparse(absolute_url)
        
        #HTMLから抽出したリンクがすでに追加されていないかを確認し、追加されていた場合はスキップする
        #追加されていない場合はto_visitに追加する
        #parsed_url.netloc == BASE_NETLOCは、リンクが対象ドメイン内であることを確認する
        if (parsed_url.netloc == BASE_NETLOC and
            #一度参照したurlをまた対象にしてしまった際に無限ループなってしまうことを防ぐために、visitedとto_visitの両方に存在しないURLのみを追加する
            absolute_url not in visited and
            absolute_url not in to_visit and
            #httpまたはhttpsスキームのURLのみを対象とする
            parsed_url.scheme in ('http', 'https')):
            
            to_visit.add(absolute_url)
            
# --- メイン実行部 ---
print(f"--- スタート: {START_URL} ---")
print(f"--- 対象: {BASE_NETLOC} ---")

try:
    while to_visit:
        # to_visitから次のURLを取得
        current_url = to_visit.pop()
        
        # ページの巡回とリンクの探索を実行
        crawl_page(current_url, scraped_data, to_visit, visited)

except KeyboardInterrupt:
    print("\n--- 中断 ---")
    

    # --- 結果の出力 ---
print("\n--- 結果 ---")
print(f"合計ページ数: {len(scraped_data)}")
print("--- 取得した辞書型変数 (scraped_data) の内容 ---")
    
# 辞書変数を表示
#URLとタイトルを見やすくするために1行ずつ表示
for url, title in scraped_data.items():
    print(f"{url} : {title}")
print(scraped_data)

--- スタート: https://www.musashino-u.ac.jp/ ---
--- 対象: www.musashino-u.ac.jp ---
https://www.musashino-u.ac.jp/
 タイトル: 武蔵野大学
https://www.musashino-u.ac.jp/contact.html
 タイトル: お問い合わせ | 武蔵野大学
https://www.musashino-u.ac.jp/student-life/life/
 タイトル: 生活サポート | 学生生活・就職 | 武蔵野大学
https://www.musashino-u.ac.jp/student-life/learning/course/faq_english.html
 タイトル: 履修・授業に関するFAQ for English | 学生生活・就職 | 武蔵野大学
https://www.musashino-u.ac.jp/academics/graduate_school/
 タイトル: 大学院 | 学部・大学院 | 武蔵野大学
https://www.musashino-u.ac.jp/basic/initial/fs/index.html
 タイトル: フィールド・スタディーズ | 武蔵野大学
https://www.musashino-u.ac.jp/research/
 タイトル: 研究 | 武蔵野大学
https://www.musashino-u.ac.jp/academics/graduate_school/course/literature/index.html
 タイトル: 文学研究科 | 学部・大学院 | 武蔵野大学
https://www.musashino-u.ac.jp/academics/graduate_school/dissertation/
 タイトル: 博士学位論文 | 学部・大学院 | 武蔵野大学
https://www.musashino-u.ac.jp/student-life/life/dining/index.html
 タイトル: 食堂 | 学生生活・就職 | 武蔵野大学
https://www.musashino-u.ac.jp/academics/graduate_school/course/pha